In [73]:
import spacy
import csv
from collections import namedtuple
from spacy.matcher import PhraseMatcher

In [74]:
o = ""

for page in range(1,60):
    with open('edited_from_ocr/Complaint-p{}-normal.txt'.format(str(page)),'r') as output:
        o = "\n".join([o, output.read()])

In [75]:
o = o.replace("\n", " ").replace("Case 1:18-cv-03501 Document 1 Filed 04/20/18 ", "")\
    .replace("$$", "§§").replace(" $ ", " § ").replace("С", "C").replace(" S ", " § ")\
    .replace(" SS", " §§").replace(" 55 "," §§ ")

In [76]:
for n in range(66):
    o = o.replace(f"Page {n} of 66", f'**{n}**')
    # o = o.replace(f"Page {n} of 66", f'<page number="{n}" />')

In [77]:
# Spacy parsing needs to be done before tags are added.

o = "<complaint>" + o + "</complaint>"

In [78]:
def tag_allegations(s: str,first: int,last: int):
    s = s.replace(f" {first}. ", f"""<allegation para="{first}">""", 1)
    for n in range(first+1, last+1):
        s = s.replace(f" {n}. ", f"""</allegation><allegation para="{n}">""", 1)
    s = s.replace(f" {last+1}. ", f"""</allegation> {last+1}. """, 1) # not good, consumes the next heading.
    return s

o = tag_allegations(o, 146, 153)

In [79]:
o

'<complaint> **1** IN THE UNITED STATES DISTRICT COURT FOR THE SOUTHERN DISTRICT OF NEW YORK DEMOCRATIC NATIONAL COMMITTEE,) Plaintiff, Civil Action No JURY DEMAND THE RUSSIAN FEDERATION GENERAL STAFF OF THE ARMED COMPLAINT COMPUTER FRAUD AND ABUSE ACT (18 U.S.C. 1030(a)) FORCES OF THE RUSSIAN FEDERATION ("GRU"); GRU OPERATIVE USING ) Rico (18 U.S.C. § 1962(c)) ) $1962(d) ) 25 10-22) ACT (18 U.S.C. §§ 2701-12) RICO CONSPIRACY (18 U.S.C WIRETAP ACT (18 U.S.C. STORED COMMUNICATIONS PSEUDONYM "GUCCIFER 2.0", ARAS ISKENEROVICH AGALAROV EMIN ARAZ AGALAROV JOSEPH MIFSUD WIKILEAKS JULIAN ASSANGE; DONALD J. TRUMP FOR PRESIDENT, INC., DONALD J. TRUMP, JR. PAUL J. MANAFORT, JR. ROGER J. STONE, JR.; JARED C. KUSHNER GEORGE PAPADOPOULOS RICHARD W. GATES, III; JOHN DOES 1-10, DIGITAL MILLENNIUM § 1201 et seq.) MISAPPROPRIATION OF TRADE SECRETS UNDER THE DEFEND TRADE SECRETS ACT (18 U.S.C § 1836 et seq.) WASHINGTON D.C. UNIFORM TRADE SECRETS ACT (D.C. Code Ann-55 36-401-46-410) ) Defendants. TRESPAS

In [6]:
parties = []

with open("./data/parties.csv", newline="") as infile:
    reader = csv.reader(infile)
    Party = namedtuple("Party", next(reader))  # get names from column headers
    for data in map(Party._make, reader):
        parties.append(data)
    

In [7]:
parties

[Party(role='Plaintiff', name='Democratic National Committee', short_name='DNC'),
 Party(role='Defendant', name='Russian Federation', short_name='Russia'),
 Party(role='Defendant', name='General Staff of the Armed Forces of the Russian Federation', short_name='GRU'),
 Party(role='Defendant', name='GRU operative using the pseudonym "Guccifer 2.0"', short_name='GRU Operative #1'),
 Party(role='Defendant', name='Ars Iskenerovich Agalarov', short_name='Aras Agalarov'),
 Party(role='Defendant', name='Emin Araz Agalarov', short_name='Emin Agalarov'),
 Party(role='Defendant', name='Joseplh Mifsud', short_name='Mifsud'),
 Party(role='Defendant', name='WikiLeaks', short_name='Wiki Leaks'),
 Party(role='Defendant', name='Julian Assange', short_name='Assange'),
 Party(role='Defendant', name='Donald J. Trump for President, Inc.', short_name='Trump Campaign'),
 Party(role='Defendant', name='Donald J. Trump, Jr.', short_name='Trump, Jr.'),
 Party(role='Defendant', name='Paul J. Manafort, Jr.', short

In [8]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(o)

In [9]:
# This is probably overkill. I should just use string methods to highlight/tag the names.

party_matches = []

def on_match(matcher, doc, uid, matches):
    print(f'{uid} matched!')
    # party_matches.append((uid, matches, "Party"))

matcher = PhraseMatcher(nlp.vocab)
for p in parties:
    if len(p.name) < 10:
        matcher.add(p.short_name, on_match, nlp(p.short_name), nlp(p.name))
    else: matcher.add(p.short_name, on_match, nlp(p.short_name))
    
# matches = matcher(doc)

In [10]:
Entity = namedtuple("Entity", "text start_char end_char label_")

In [11]:
entities = [Entity(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents
            if e.label_ not in ("CARDINAL", "ORDINAL", "DATE", "MONEY", "TIME", "LAW", "FAC")]

In [12]:
entities

[Entity(text=' Page 1 of 66', start_char=0, end_char=13, label_='ORG'),
 Entity(text='THE UNITED STATES DISTRICT COURT', start_char=17, end_char=49, label_='GPE'),
 Entity(text='THE SOUTHERN DISTRICT OF NEW YORK DEMOCRATIC NATIONAL COMMITTEE', start_char=54, end_char=117, label_='WORK_OF_ART'),
 Entity(text='RUSSIAN', start_char=163, end_char=170, label_='NORP'),
 Entity(text='THE ARMED COMPLAINT COMPUTER', start_char=199, end_char=227, label_='ORG'),
 Entity(text='U.S.C.', start_char=252, end_char=258, label_='GPE'),
 Entity(text='FORCES OF THE RUSSIAN FEDERATION', start_char=268, end_char=300, label_='ORG'),
 Entity(text='GRU', start_char=303, end_char=306, label_='ORG'),
 Entity(text='GRU OPERATIVE', start_char=310, end_char=323, label_='ORG'),
 Entity(text='Rico', start_char=332, end_char=336, label_='PERSON'),
 Entity(text='U.S.C.', start_char=341, end_char=347, label_='GPE'),
 Entity(text='ARAS ISKENEROVICH AGALAROV', start_char=506, end_char=532, label_='ORG'),
 Entity(text='JOS

In [13]:
for sent in list(doc.sents)[100:110]:
    print(sent.text)

In 2015 and 2016, Russian intelligence services hacked into the DNC's computers, penetrated its phone systems, and exfiltrated tens of thousands of documents and emails.
Russia then used this stolen information to advance its own interests: destabilizing the U.S. political environment, denigrating the Democratic presidential nominee, and supporting the campaign of Donald J. Trump ("Trump"), whose policies would benefit the Kremlin.
2
In the Trump campaign, Russia found a willing and active partner in this effort.
In 2016, individuals tied to the Kremlin notified the Trump campaign that Russia intended to interfere with our democracy.
Through multiple meetings, emails, and other communications, these Russian agents made clear that their government supported Trump and was prepared to use stolen emails and other information to damage his opponent and the Democratic party.  
Page 6 of 66 3.
Rather than report these repeated messages that Russia intended to interfere with U.S. elections, th

In [14]:
doc.ents[1]

THE UNITED STATES DISTRICT COURT

In [15]:
type(doc.ents[1])

spacy.tokens.span.Span

In [16]:
doc.ents

( Page 1 of 66,
 THE UNITED STATES DISTRICT COURT,
 THE SOUTHERN DISTRICT OF NEW YORK DEMOCRATIC NATIONAL COMMITTEE,
 RUSSIAN,
 THE ARMED COMPLAINT COMPUTER,
 18,
 U.S.C.,
 FORCES OF THE RUSSIAN FEDERATION,
 GRU,
 GRU OPERATIVE,
 Rico,
 18,
 U.S.C.,
 1962(d,
 25,
 18 U.S.C. $$ 2701-12,
 RICO,
 18,
 U.S.C WIRETAP ACT,
 18,
 2.0,
 ARAS ISKENEROVICH AGALAROV,
 JOSEPH MIFSUD,
 DONALD J. TRUMP,
 INC,
 DONALD J. TRUMP,
 JR,
 PAUL J. MANAFORT,
 JR,
 ROGER J. STONE,
 JR,
 JARED C. KUSHNER,
 RICHARD W. GATES,
 JOHN,
 1-10,
 DIGITAL MILLENNIUM,
 1201,
 THE DEFEND TRADE SECRETS ACT,
 18,
 U.S.C,
 1836,
 WASHINGTON,
 D.C.,
 Ann-55,
 36-401,
 TRESPASS,
 CHATTELS,
 Virginia Common Law),
 CHATTELS,
 Virginia Common Law,
 VIRGINIA COMPUTER CRIMES,
 Va.,
 Code Ann.18.2-152.5 et seq,
 2,
 66,
 DNC,
 VENUE,
 TRUMP ASSOCIATES',
 PRE-EXISTING RELATIONSHIPS,
 13,
 RUSSIA,
 RUSSIAN,
 RUSSIA,
 15,
 DNC,
 17,
 TRUMP A,
 Trump Announces His,
 Russia,
 Begins,
 18,
 B. European,
 U.S.,
 Trump Signs a Letter of I

In [17]:
import xml.etree.ElementTree as ET
root = ET.fromstring(o)

ParseError: syntax error: line 1, column 1 (<string>)